# Enzymes dataset benchmark

Paper  "Protein function prediction via graph kernels" , 91-95% accuracy on one-vs-rest classification of each class. No F1-scores

## References
- [4] K. M. Borgwardt, C. S. Ong, S. Schoenauer, S. V. N. Vishwanathan, A. J. Smola, and H. P. Kriegel. Protein function prediction via graph kernels. Bioinformatics, 21(Suppl 1):i47–i56, Jun 2005.
- [5] I. Schomburg, A. Chang, C. Ebeling, M. Gremse, C. Heldt, G. Huhn, and D. Schomburg. Brenda, the enzyme database: updates and major new developments. Nucleic Acids Research, 32D:431–433, 2004.

## Description
This notebook will experiment with:
- train & compare differnt GNN models for graph classificaiton in common benchmarks (Enzymes)
- compare results to publication results



## Model training experiments

In [1]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import MessagePassing
#from torch_geometric.nn.conv.gated_graph_conv import GatedGraphConv
from torch_geometric.nn.glob.glob import global_mean_pool, global_add_pool
import torch.nn as nn
from torch.nn import Sequential as Seq, Linear as Lin, ReLU
from torch_scatter import scatter_mean
from torch_geometric.nn import MetaLayer
from torch_geometric.datasets import TUDataset, QM9, QM7b, PPI, Planetoid, KarateClub
import copy

from TFM_graph_classification import *
from TFM_graph_classification_models import *

global device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [2]:
dataset = TUDataset(root='/tmp/ENZYMES', name='ENZYMES')
dataset = dataset.shuffle()
num_classes=dataset.num_classes

k=3

train_dataset, test_dataset = prepare_dataset(dataset, nfolds=k, prop=0.8, 
                                              dataset_type="balanced", print_debug=True)

 n: 600  k folds= 3
Datasets balancing: 
{0: 100, 1: 100, 2: 100, 3: 100, 4: 100, 5: 100}
{0: 80, 1: 80, 2: 80, 3: 80, 4: 80, 5: 80}
{0: 20, 1: 20, 2: 20, 3: 20, 4: 20, 5: 20}



In [3]:
resultsdict = {'best_models_list':[], 'models':{} , 'best_models':{}, 'autoincrement': 0 }

In [4]:
model_list21 = hpsearch(None, {
    'd1': [25,50],
    'd2': [20,50],
    'epochs': [200,400],
    'model': [GGNN1],
    'learning_rate': [0.01],
    'weight_decay':[5e-4],
    'num_layers':[2,4,8],
    'aggr_type': ['mean','add'],
    'batch_size': [32]
})
modelsdict1 = modelSelection(model_list21,k, train_dataset, balanced=False)
reportModelSelectionResult(modelsdict1,resultsdict)

 trained model:  GGNN1 {'d1': 25, 'd2': 20, 'num_layers': 2, 'aggr_type': 'mean'}  epochs: 200  val loss= 0.05161553497115771  val accuracy= 0.3452380952380952  val microF1= 0.3782051282051282  val macroF1= 0.3810069446654874
 trained model:  GGNN1 {'d1': 25, 'd2': 20, 'num_layers': 2, 'aggr_type': 'add'}  epochs: 200  val loss= 0.04935483386119207  val accuracy= 0.4880952380952381  val microF1= 0.4594017094017094  val macroF1= 0.46062061338596977
 trained model:  GGNN1 {'d1': 25, 'd2': 20, 'num_layers': 4, 'aggr_type': 'mean'}  epochs: 200  val loss= 0.05324504400293032  val accuracy= 0.4047619047619047  val microF1= 0.35683760683760685  val macroF1= 0.3592613536149005
 trained model:  GGNN1 {'d1': 25, 'd2': 20, 'num_layers': 4, 'aggr_type': 'add'}  epochs: 200  val loss= 0.057577989995479584  val accuracy= 0.4047619047619048  val microF1= 0.38247863247863245  val macroF1= 0.39628611286456294
 trained model:  GGNN1 {'d1': 25, 'd2': 20, 'num_layers': 8, 'aggr_type': 'mean'}  epochs: 20

 trained model:  GGNN1 {'d1': 50, 'd2': 50, 'num_layers': 2, 'aggr_type': 'add'}  epochs: 200  val loss= 0.05352921038866043  val accuracy= 0.4166666666666667  val microF1= 0.4551282051282051  val macroF1= 0.46120655709826114
 trained model:  GGNN1 {'d1': 50, 'd2': 50, 'num_layers': 4, 'aggr_type': 'mean'}  epochs: 200  val loss= 0.054165687412023544  val accuracy= 0.38095238095238093  val microF1= 0.35897435897435903  val macroF1= 0.3642508151993855
 trained model:  GGNN1 {'d1': 50, 'd2': 50, 'num_layers': 4, 'aggr_type': 'add'}  epochs: 200  val loss= 0.0545924020310243  val accuracy= 0.25  val microF1= 0.3269230769230769  val macroF1= 0.33423565393984855
 trained model:  GGNN1 {'d1': 50, 'd2': 50, 'num_layers': 8, 'aggr_type': 'mean'}  epochs: 200  val loss= 0.057428342600663505  val accuracy= 0.16666666666666666  val microF1= 0.16666666666666666  val macroF1= 0.04761904761904762
 trained model:  GGNN1 {'d1': 50, 'd2': 50, 'num_layers': 8, 'aggr_type': 'add'}  epochs: 200  val loss=

,best_model_loss,best_model_acc,best_model_microF1,best_model_macroF1
accuracy,0.571429,0.714286,0.714286,0.714286
batch_size,32,32,32,32
cv_val_accuracy,0.488095,0.559524,0.559524,0.559524
cv_val_loss,0.0493548,0.0553561,0.0553561,0.0553561
cv_val_macroF1,0.460621,0.516487,0.516487,0.516487
cv_val_microF1,0.459402,0.510684,0.510684,0.510684
epochs,200,400,400,400
filename,0_GGNN1__d1-25_d2-20_num_layers-2_aggr_type-ad...,3_GGNN1__d1-25_d2-50_num_layers-2_aggr_type-ad...,3_GGNN1__d1-25_d2-50_num_layers-2_aggr_type-ad...,3_GGNN1__d1-25_d2-50_num_layers-2_aggr_type-ad...
final_model,"GGNN1(\n (ggnn): GatedGraphConv(25, num_layer...","GGNN1(\n (ggnn): GatedGraphConv(25, num_layer...","GGNN1(\n (ggnn): GatedGraphConv(25, num_layer...","GGNN1(\n (ggnn): GatedGraphConv(25, num_layer..."
kwargs,"{'d1': 25, 'd2': 20, 'num_layers': 2, 'aggr_ty...","{'d1': 25, 'd2': 50, 'num_layers': 2, 'aggr_ty...","{'d1': 25, 'd2': 50, 'num_layers': 2, 'aggr_ty...","{'d1': 25, 'd2': 50, 'num_layers': 2, 'aggr_ty..."


In [5]:
model_list22 = hpsearch(None, {
    'model': [GGNN2],
    'd1': [25,50],
    'd2': [20,50],
    'aggr_type': ['mean','add'],
    'epochs': [300,600],
    'num_layers':[2,4,8],
    'learning_rate': [0.01],
    'weight_decay':[5e-4],
    'batch_size': [32]
})
modelsdict2 = modelSelection(model_list22,k, train_dataset)
reportModelSelectionResult(modelsdict2,resultsdict)

 trained model:  GGNN2 {'d1': 25, 'd2': 20, 'aggr_type': 'mean', 'num_layers': 2}  epochs: 300  val loss= 0.05107565348347028  val accuracy= 0.48809523809523814  val microF1= 0.44017094017094016  val macroF1= 0.44259182222886534
 trained model:  GGNN2 {'d1': 25, 'd2': 20, 'aggr_type': 'mean', 'num_layers': 4}  epochs: 300  val loss= 0.06114205593864123  val accuracy= 0.36904761904761907  val microF1= 0.40598290598290604  val macroF1= 0.40636680523236485
 trained model:  GGNN2 {'d1': 25, 'd2': 20, 'aggr_type': 'mean', 'num_layers': 8}  epochs: 300  val loss= 0.056891450037558876  val accuracy= 0.2380952380952381  val microF1= 0.23290598290598288  val macroF1= 0.21834857015495843
 trained model:  GGNN2 {'d1': 25, 'd2': 20, 'aggr_type': 'mean', 'num_layers': 2}  epochs: 600  val loss= 0.051027391105890274  val accuracy= 0.4166666666666667  val microF1= 0.4252136752136752  val macroF1= 0.4362993037068826
 trained model:  GGNN2 {'d1': 25, 'd2': 20, 'aggr_type': 'mean', 'num_layers': 4}  epo

 trained model:  GGNN2 {'d1': 50, 'd2': 50, 'aggr_type': 'mean', 'num_layers': 4}  epochs: 300  val loss= 0.05588945125540098  val accuracy= 0.25  val microF1= 0.27991452991452986  val macroF1= 0.2845206245574803
 trained model:  GGNN2 {'d1': 50, 'd2': 50, 'aggr_type': 'mean', 'num_layers': 8}  epochs: 300  val loss= 0.05741229529182116  val accuracy= 0.17857142857142858  val microF1= 0.17521367521367523  val macroF1= 0.0787554768971726
 trained model:  GGNN2 {'d1': 50, 'd2': 50, 'aggr_type': 'mean', 'num_layers': 2}  epochs: 600  val loss= 0.05200367917617162  val accuracy= 0.42857142857142855  val microF1= 0.46581196581196577  val macroF1= 0.4691060381951202
 trained model:  GGNN2 {'d1': 50, 'd2': 50, 'aggr_type': 'mean', 'num_layers': 4}  epochs: 600  val loss= 0.0643631840745608  val accuracy= 0.28571428571428575  val microF1= 0.31196581196581197  val macroF1= 0.31923371000680906
 trained model:  GGNN2 {'d1': 50, 'd2': 50, 'aggr_type': 'mean', 'num_layers': 8}  epochs: 600  val los

,best_model_loss,best_model_acc,best_model_microF1,best_model_macroF1
accuracy,0.535714,0.571429,0.428571,0.428571
batch_size,32,32,32,32
cv_val_accuracy,0.464286,0.52381,0.488095,0.488095
cv_val_loss,0.0492011,0.0500142,0.0534327,0.0534327
cv_val_macroF1,0.438439,0.474731,0.488783,0.488783
cv_val_microF1,0.435897,0.472222,0.478632,0.478632
epochs,300,300,600,600
filename,4_GGNN2__d1-25_d2-50_aggr_type-add_num_layers-...,5_GGNN2__d1-50_d2-20_aggr_type-add_num_layers-...,7_GGNN2__d1-50_d2-20_aggr_type-mean_num_layers...,7_GGNN2__d1-50_d2-20_aggr_type-mean_num_layers...
final_model,"GGNN2(\n (ggnn): GatedGraphConv(25, num_layer...","GGNN2(\n (ggnn): GatedGraphConv(50, num_layer...","GGNN2(\n (ggnn): GatedGraphConv(50, num_layer...","GGNN2(\n (ggnn): GatedGraphConv(50, num_layer..."
kwargs,"{'d1': 25, 'd2': 50, 'aggr_type': 'add', 'num_...","{'d1': 50, 'd2': 20, 'aggr_type': 'add', 'num_...","{'d1': 50, 'd2': 20, 'aggr_type': 'mean', 'num...","{'d1': 50, 'd2': 20, 'aggr_type': 'mean', 'num..."


In [6]:
model_list23 = hpsearch(None, {
    'model': [GGNN3],
    'd1': [25,50],
    'd2': [20,50],
    'aggr_type': ['mean','add'],
    'epochs': [200,400],
    'num_layers':[2,4,8],
    'learning_rate': [0.01],
    'weight_decay':[5e-4],
    'batch_size': [32]
})
modelsdict3 = modelSelection(model_list23,k, train_dataset)
reportModelSelectionResult(modelsdict3,resultsdict)

 trained model:  GGNN3 {'d1': 25, 'd2': 20, 'aggr_type': 'mean', 'num_layers': 2}  epochs: 200  val loss= 0.05328625068068504  val accuracy= 0.6190476190476191  val microF1= 0.48504273504273504  val macroF1= 0.48554875746269577
 trained model:  GGNN3 {'d1': 25, 'd2': 20, 'aggr_type': 'mean', 'num_layers': 4}  epochs: 200  val loss= 0.051970151563485466  val accuracy= 0.46428571428571425  val microF1= 0.4017094017094016  val macroF1= 0.40421924946230653
 trained model:  GGNN3 {'d1': 25, 'd2': 20, 'aggr_type': 'mean', 'num_layers': 8}  epochs: 200  val loss= 0.057101067155599594  val accuracy= 0.17857142857142858  val microF1= 0.1794871794871795  val macroF1= 0.1090230688736251
 trained model:  GGNN3 {'d1': 25, 'd2': 20, 'aggr_type': 'mean', 'num_layers': 2}  epochs: 400  val loss= 0.057025584081808724  val accuracy= 0.42857142857142855  val microF1= 0.47008547008547014  val macroF1= 0.46827780842262673
 trained model:  GGNN3 {'d1': 25, 'd2': 20, 'aggr_type': 'mean', 'num_layers': 4}  ep

 trained model:  GGNN3 {'d1': 50, 'd2': 50, 'aggr_type': 'mean', 'num_layers': 4}  epochs: 200  val loss= 0.058606033523877464  val accuracy= 0.2738095238095238  val microF1= 0.22863247863247863  val macroF1= 0.2262655352669719
 trained model:  GGNN3 {'d1': 50, 'd2': 50, 'aggr_type': 'mean', 'num_layers': 8}  epochs: 200  val loss= 0.06122704967856407  val accuracy= 0.11904761904761903  val microF1= 0.16666666666666666  val macroF1= 0.04770674383934053
 trained model:  GGNN3 {'d1': 50, 'd2': 50, 'aggr_type': 'mean', 'num_layers': 2}  epochs: 400  val loss= 0.06119890883564949  val accuracy= 0.4166666666666667  val microF1= 0.4764957264957266  val macroF1= 0.47679899466506637
 trained model:  GGNN3 {'d1': 50, 'd2': 50, 'aggr_type': 'mean', 'num_layers': 4}  epochs: 400  val loss= 0.052997760474681854  val accuracy= 0.2738095238095238  val microF1= 0.34401709401709396  val macroF1= 0.35178465931401565
 trained model:  GGNN3 {'d1': 50, 'd2': 50, 'aggr_type': 'mean', 'num_layers': 8}  epoc

,best_model_loss,best_model_acc,best_model_microF1,best_model_macroF1
accuracy,0.535714,0.607143,0.571429,0.571429
batch_size,32,32,32,32
cv_val_accuracy,0.452381,0.619048,0.52381,0.52381
cv_val_loss,0.0508016,0.0532863,0.057745,0.057745
cv_val_macroF1,0.444888,0.485549,0.505192,0.505192
cv_val_microF1,0.440171,0.485043,0.5,0.5
epochs,200,200,400,400
filename,8_GGNN3__d1-50_d2-50_aggr_type-add_num_layers-...,9_GGNN3__d1-25_d2-20_aggr_type-mean_num_layers...,11_GGNN3__d1-50_d2-50_aggr_type-add_num_layers...,11_GGNN3__d1-50_d2-50_aggr_type-add_num_layers...
final_model,"GGNN3(\n (ggnn): GatedGraphConv(50, num_layer...","GGNN3(\n (ggnn): GatedGraphConv(25, num_layer...","GGNN3(\n (ggnn): GatedGraphConv(50, num_layer...","GGNN3(\n (ggnn): GatedGraphConv(50, num_layer..."
kwargs,"{'d1': 50, 'd2': 50, 'aggr_type': 'add', 'num_...","{'d1': 25, 'd2': 20, 'aggr_type': 'mean', 'num...","{'d1': 50, 'd2': 50, 'aggr_type': 'add', 'num_...","{'d1': 50, 'd2': 50, 'aggr_type': 'add', 'num_..."


In [7]:
model_list24 = hpsearch(None, {
    'model': [GGNN4],
    'd1': [25,50],
    'd2': [20,50],
    'aggr_type': ['mean','add'],
    'epochs': [200,400],
    'num_layers':[2,4,8],
    'learning_rate': [0.01],
    'weight_decay':[5e-4],
    'batch_size': [32]
})
modelsdict4 = modelSelection(model_list24,k, train_dataset)
reportModelSelectionResult(modelsdict4,resultsdict)

 trained model:  GGNN4 {'d1': 25, 'd2': 20, 'aggr_type': 'mean', 'num_layers': 2}  epochs: 200  val loss= 0.05592741072177887  val accuracy= 0.4880952380952381  val microF1= 0.48290598290598297  val macroF1= 0.4849658894042997
 trained model:  GGNN4 {'d1': 25, 'd2': 20, 'aggr_type': 'mean', 'num_layers': 4}  epochs: 200  val loss= 0.05359400932987531  val accuracy= 0.4404761904761905  val microF1= 0.42094017094017094  val macroF1= 0.42501124764097825
 trained model:  GGNN4 {'d1': 25, 'd2': 20, 'aggr_type': 'mean', 'num_layers': 8}  epochs: 200  val loss= 0.055060007919867836  val accuracy= 0.2976190476190476  val microF1= 0.26068376068376065  val macroF1= 0.22471214127488856
 trained model:  GGNN4 {'d1': 25, 'd2': 20, 'aggr_type': 'mean', 'num_layers': 2}  epochs: 400  val loss= 0.0558168056110541  val accuracy= 0.39285714285714285  val microF1= 0.5064102564102564  val macroF1= 0.5129117104360191
 trained model:  GGNN4 {'d1': 25, 'd2': 20, 'aggr_type': 'mean', 'num_layers': 4}  epochs:

 trained model:  GGNN4 {'d1': 50, 'd2': 50, 'aggr_type': 'mean', 'num_layers': 4}  epochs: 200  val loss= 0.05254355072975159  val accuracy= 0.3333333333333333  val microF1= 0.34829059829059833  val macroF1= 0.3639523234538025
 trained model:  GGNN4 {'d1': 50, 'd2': 50, 'aggr_type': 'mean', 'num_layers': 8}  epochs: 200  val loss= 0.05856581528981527  val accuracy= 0.17857142857142858  val microF1= 0.16452991452991453  val macroF1= 0.047092870297290194
 trained model:  GGNN4 {'d1': 50, 'd2': 50, 'aggr_type': 'mean', 'num_layers': 2}  epochs: 400  val loss= 0.06046704823772112  val accuracy= 0.5714285714285715  val microF1= 0.5277777777777778  val macroF1= 0.536429661472182
 trained model:  GGNN4 {'d1': 50, 'd2': 50, 'aggr_type': 'mean', 'num_layers': 4}  epochs: 400  val loss= 0.05459257960319519  val accuracy= 0.3452380952380952  val microF1= 0.31837606837606836  val macroF1= 0.3225630559757391
 trained model:  GGNN4 {'d1': 50, 'd2': 50, 'aggr_type': 'mean', 'num_layers': 8}  epochs: 

,best_model_loss,best_model_acc,best_model_microF1,best_model_macroF1
accuracy,0.607143,0.607143,0.607143,0.607143
batch_size,32,32,32,32
cv_val_accuracy,0.5,0.571429,0.5,0.5
cv_val_loss,0.0521404,0.060467,0.0521404,0.0521404
cv_val_macroF1,0.559628,0.53643,0.559628,0.559628
cv_val_microF1,0.553419,0.527778,0.553419,0.553419
epochs,400,400,400,400
filename,15_GGNN4__d1-25_d2-50_aggr_type-mean_num_layer...,13_GGNN4__d1-50_d2-50_aggr_type-mean_num_layer...,15_GGNN4__d1-25_d2-50_aggr_type-mean_num_layer...,15_GGNN4__d1-25_d2-50_aggr_type-mean_num_layer...
final_model,"GGNN4(\n (ggnn): GatedGraphConv(25, num_layer...","GGNN4(\n (ggnn): GatedGraphConv(50, num_layer...","GGNN4(\n (ggnn): GatedGraphConv(25, num_layer...","GGNN4(\n (ggnn): GatedGraphConv(25, num_layer..."
kwargs,"{'d1': 25, 'd2': 50, 'aggr_type': 'mean', 'num...","{'d1': 50, 'd2': 50, 'aggr_type': 'mean', 'num...","{'d1': 25, 'd2': 50, 'aggr_type': 'mean', 'num...","{'d1': 25, 'd2': 50, 'aggr_type': 'mean', 'num..."


In [8]:
model_list25 = hpsearch(None, {
    'model': [META3,],
    'd1': [3],
    'd2': [10,20,50],
    'd3':[15],
    'd4':[15],
    'd5':[10,20],
    'epochs': [200,400],
    'learning_rate': [0.01],
    'weight_decay':[5e-4],
    'batch_size': [64],
    'num_classes': [num_classes]
})
modelsdict5 = modelSelection(model_list25,k, train_dataset)
reportModelSelectionResult(modelsdict5,resultsdict)

 trained model:  META3 {'d1': 3, 'd2': 10, 'd3': 15, 'd4': 15, 'd5': 10, 'num_classes': 6}  epochs: 200  val loss= 0.03134127830465635  val accuracy= 0.38095238095238093  val microF1= 0.3376068376068376  val macroF1= 0.33976369388517963
 trained model:  META3 {'d1': 3, 'd2': 10, 'd3': 15, 'd4': 15, 'd5': 10, 'num_classes': 6}  epochs: 400  val loss= 0.030583721896012623  val accuracy= 0.3928571428571428  val microF1= 0.391025641025641  val macroF1= 0.4073889795990409
 trained model:  META3 {'d1': 3, 'd2': 10, 'd3': 15, 'd4': 15, 'd5': 20, 'num_classes': 6}  epochs: 200  val loss= 0.02980109676718712  val accuracy= 0.44047619047619047  val microF1= 0.3952991452991453  val macroF1= 0.3883170957283522
 trained model:  META3 {'d1': 3, 'd2': 10, 'd3': 15, 'd4': 15, 'd5': 20, 'num_classes': 6}  epochs: 400  val loss= 0.030651236573855083  val accuracy= 0.32142857142857145  val microF1= 0.4038461538461539  val macroF1= 0.40482107011542845
 trained model:  META3 {'d1': 3, 'd2': 20, 'd3': 15, '

,best_model_loss,best_model_acc,best_model_microF1,best_model_macroF1
accuracy,0.428571,0.464286,0.464286,0.464286
batch_size,64,64,64,64
cv_val_accuracy,0.428571,0.511905,0.416667,0.416667
cv_val_loss,0.0295395,0.0298069,0.0299006,0.0299006
cv_val_macroF1,0.42113,0.413736,0.440353,0.440353
cv_val_microF1,0.42094,0.410256,0.438034,0.438034
epochs,400,200,200,200
filename,16_META3__d1-3_d2-50_d3-15_d4-15_d5-20_num_cla...,17_META3__d1-3_d2-20_d3-15_d4-15_d5-10_num_cla...,19_META3__d1-3_d2-50_d3-15_d4-15_d5-20_num_cla...,19_META3__d1-3_d2-50_d3-15_d4-15_d5-20_num_cla...
final_model,META3(\n (edge_mlp): Sequential(\n (0): Li...,META3(\n (edge_mlp): Sequential(\n (0): Li...,META3(\n (edge_mlp): Sequential(\n (0): Li...,META3(\n (edge_mlp): Sequential(\n (0): Li...
kwargs,"{'d1': 3, 'd2': 50, 'd3': 15, 'd4': 15, 'd5': ...","{'d1': 3, 'd2': 20, 'd3': 15, 'd4': 15, 'd5': ...","{'d1': 3, 'd2': 50, 'd3': 15, 'd4': 15, 'd5': ...","{'d1': 3, 'd2': 50, 'd3': 15, 'd4': 15, 'd5': ..."


In [9]:
model_list26 = hpsearch(None, {
    'model': [META2,],
    'd1': [3],
    'd2': [10,20,50],
    'd3':[15],
    'd4':[15],
    'd5':[10,20],
    'epochs': [200,400],
    'learning_rate': [0.01],
    'weight_decay':[5e-4],
    'batch_size': [64],
    'num_classes': [num_classes]
})
modelsdict6 = modelSelection(model_list26,k, train_dataset, debug_training=False)
reportModelSelectionResult(modelsdict6, resultsdict)


 selected model from loss:  20_META2 {'d1': 3, 'd2': 50, 'd3': 15, 'd4': 15, 'd5': 10, 'num_classes': 6}  epochs: 200 0.03672536090016365 0.2857142857142857 0.25 0.28133647794059885
 selected model from accuracy:  21_META2 {'d1': 3, 'd2': 10, 'd3': 15, 'd4': 15, 'd5': 10, 'num_classes': 6}  epochs: 200 0.041939251124858856 0.3452380952380952 0.25 0.28133647794059885
 selected model from microF1:  22_META2 {'d1': 3, 'd2': 50, 'd3': 15, 'd4': 15, 'd5': 10, 'num_classes': 6}  epochs: 400 0.036991539100805916 0.2976190476190476 0.297008547008547 0.30055438201883317
 selected model from macroF1:  23_META2 {'d1': 3, 'd2': 10, 'd3': 15, 'd4': 15, 'd5': 20, 'num_classes': 6}  epochs: 400 0.06768510987361272 0.2738095238095238 0.2905982905982906 0.32045694190754853


,best_model_loss,best_model_acc,best_model_microF1,best_model_macroF1
accuracy,0.285714,0.464286,0.285714,0.357143
batch_size,64,64,64,64
cv_val_accuracy,0.285714,0.345238,0.297619,0.27381
cv_val_loss,0.0367254,0.0419393,0.0369915,0.0676851
cv_val_macroF1,0.281336,0.26934,0.300554,0.320457
cv_val_microF1,0.25,0.258547,0.297009,0.290598
epochs,200,200,400,400
filename,20_META2__d1-3_d2-50_d3-15_d4-15_d5-10_num_cla...,21_META2__d1-3_d2-10_d3-15_d4-15_d5-10_num_cla...,22_META2__d1-3_d2-50_d3-15_d4-15_d5-10_num_cla...,23_META2__d1-3_d2-10_d3-15_d4-15_d5-20_num_cla...
final_model,META2(\n (edge_mlp): Sequential(\n (0): Li...,META2(\n (edge_mlp): Sequential(\n (0): Li...,META2(\n (edge_mlp): Sequential(\n (0): Li...,META2(\n (edge_mlp): Sequential(\n (0): Li...
kwargs,"{'d1': 3, 'd2': 50, 'd3': 15, 'd4': 15, 'd5': ...","{'d1': 3, 'd2': 10, 'd3': 15, 'd4': 15, 'd5': ...","{'d1': 3, 'd2': 50, 'd3': 15, 'd4': 15, 'd5': ...","{'d1': 3, 'd2': 10, 'd3': 15, 'd4': 15, 'd5': ..."


## Testing

In [10]:
test_multiple_models(resultsdict, test_dataset)

23_META2
23_META2__d1-3_d2-10_d3-15_d4-15_d5-20_num_classes-6_400_0.01_0.0005_64_date2019-06-27_04-34-15
20_META2
20_META2__d1-3_d2-50_d3-15_d4-15_d5-10_num_classes-6_200_0.01_0.0005_64_date2019-06-27_04-34-15
22_META2
22_META2__d1-3_d2-50_d3-15_d4-15_d5-10_num_classes-6_400_0.01_0.0005_64_date2019-06-27_04-34-15
21_META2
21_META2__d1-3_d2-10_d3-15_d4-15_d5-10_num_classes-6_200_0.01_0.0005_64_date2019-06-27_04-34-15
19_META3
19_META3__d1-3_d2-50_d3-15_d4-15_d5-20_num_classes-6_200_0.01_0.0005_64_date2019-06-27_04-24-52
19_META3
19_META3__d1-3_d2-50_d3-15_d4-15_d5-20_num_classes-6_200_0.01_0.0005_64_date2019-06-27_04-24-52
16_META3
16_META3__d1-3_d2-50_d3-15_d4-15_d5-20_num_classes-6_400_0.01_0.0005_64_date2019-06-27_04-24-52
8_GGNN3
8_GGNN3__d1-50_d2-50_aggr_type-add_num_layers-2_200_0.01_0.0005_32_date2019-06-27_02-47-57
4_GGNN2
4_GGNN2__d1-25_d2-50_aggr_type-add_num_layers-2_300_0.01_0.0005_32_date2019-06-27_01-50-08
0_GGNN1
0_GGNN1__d1-25_d2-20_num_layers-2_aggr_type-add_200_0.01_0.

## Final results report
We have obtained a **?%** to **?%** accuracy on dataset with published accuracy of a **72.33%** to **84.08%**.


In [11]:
report_all_test(resultsdict)

2019-06-27_04-34-16


,accuracy,macroF1,microF1,name
1,0.2417,0.2207,0.2417,20_META2
0,0.2833,0.2975,0.2833,23_META2
2,0.3333,0.3050,0.3333,22_META2
3,0.3333,0.3240,0.3333,21_META2
11,0.3417,0.3610,0.3417,17_META3
8,0.3417,0.3276,0.3417,0_GGNN1
4,0.3750,0.3734,0.3750,19_META3
12,0.3750,0.3872,0.3750,5_GGNN2
16,0.4000,0.3957,0.4000,9_GGNN3
9,0.4083,0.4068,0.4083,7_GGNN2


## Reload models from disk

In [3]:
filenames =[
    './models/enzymes2/0_GGNN1__d1-25_d2-50_num_layers-4_aggr_type-add_300_0.01_0.0005_32_date2019-06-26_01-15-24',
]

#resultsdict_old = copy.deepcopy(resultsdict)
resultsdict = {'best_models_list':[], 'models':[] , 'best_models':{}, 'testing': {}, 'autoincrement': 0 }
for path in filenames:
    modeldict = loadModelFromFile(path)
    resultsdict['models'].append(modeldict)
    resultsdict['best_models_list'].append(modeldict)


In [1]:
test_multiple_models(resultsdict, test_dataset)
report_all_test(resultsdict)